# Replica project

### Iconographic subset

Creates subset, stores pre-pooling embeddings for items in subset and for 5 most similar ones to each using embeddings, attempts clustering and mapper.

In [26]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from scipy import sparse
import pickle
import sys

sys.path.insert(0, "../model/")
from utils import *

path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir = '/mnt/project_replica/datasets/cini/'
wga_dir = '/mnt/project_replica/datasets/WGA/'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
#metadata['path'] = metadata['Drawer'].astype(str) + '/' + metadata['Drawer'].astype(str) + '_' + metadata['ImageNumber'].astype(str) + '.jpg'
metadata = pd.read_csv(data_dir + 'dedup_data.csv').drop(columns=['Unnamed: 0', 'level_0'])
metadata.head(5)

/tmp/ipykernel_3854251/2466698109.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(data_dir + 'dedup_data.csv').drop(columns=['Unnamed: 0', 'level_0'])


,uid,uid_connection,cluster,index,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,...,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,uid2,set,path
0,00000763d00845d7aa8f796fa3e86e20,c8ac0f0ea2524e74a8b90c92138260a5,4041,0,61A,632,MARESCALCHI Pietro,Madonna in trono e Santi.,"MAKESCALCHI, Pietro.","Marascalchi, Pietro de'",...,Feltre,https://dhlabsrv4.epfl.ch/iiif_cini/61A%2F61A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,00000763d00845d7aa8f796fa3e86e20,3e1be39c07bd4203848d7ebe44db3528,DUPLICATE,2018-07-19 17:17:46.960437+00:00,61A/61A_632.jpg,NaN,61A/61A_632.jpg
1,00001c888748454ea4ecf8997f902f1f,Not uid connection,00001c888748454ea4ecf8997f902f1f,3,87A,507,SCULTORE SEC XVII,Colonna . Part.,SCULTORE SEC. XVII,NaN,...,NaN,https://dhlabsrv4.epfl.ch/iiif_cini/87A%2F87A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaN,87A/87A_507.jpg,NaN,87A/87A_507.jpg
2,000036a6440344f89e0f0e120e962f88,Not uid connection,000036a6440344f89e0f0e120e962f88,4,1C,484,TIZIANO Vecellio,Giovane guerriero. 272x178,TIZIANO.,Titian,...,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/1C%2F1C_48...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaN,1C/1C_484.jpg,NaN,1C/1C_484.jpg
3,000092c815b6453eb6f37ccabbc8689d,Not uid connection,000092c815b6453eb6f37ccabbc8689d,5,42C,694,BOSCHINI Marco,Pag. 664,"BOSCHINI, Marco.","Boschini, Marco",...,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/42C%2F42C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaN,42C/42C_694.jpg,NaN,42C/42C_694.jpg
4,0000b1a1cf3843619edb853fa03a2d5d,Not uid connection,0000b1a1cf3843619edb853fa03a2d5d,6,96A,336,CATTANEO Danese,Allegoria.,"CATTANEO, Danese.","Cattaneo, Danese",...,NaN,https://dhlabsrv4.epfl.ch/iiif_cini/96A%2F96A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaN,96A/96A_336.jpg,NaN,96A/96A_336.jpg


In [12]:
# in jeanne's code
# with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
#     morpho_graph_complete = pickle.load(f)

# # contains uid	img1	img2	type	annotated
# morpho_graph_complete.head()

In [13]:
madonnas = metadata[metadata['Description'].fillna('').str.contains('Madonna col Bambino')]#['Description']
madonnas.shape

(2074, 47)

In [15]:
# positives = get_train_test_split(madonnas, morpho_graph_complete)
# positives = positives.groupby('uid_connection').first().reset_index()
# print(positives.shape)
# positives.head()

In [18]:
train = madonnas[madonnas['set'].isin(['train', 'val', 'test'])]
train.shape

(274, 47)

## Getting set of images to store embeddings of

In [19]:
embeddings = np.load(
                    #data_dir + 'resnext-101_avg_480' + ".npy",
                    data_dir + 'embeddings/resnext-101_epoch_314-04-2022_23:25:29.npy',
                    allow_pickle=True,
                )        

In [21]:
tree, reverse_map = make_tree_orig(embeddings, reverse_map=True)

In [22]:
with open(data_dir + 'uid2path.pkl', 'rb') as outfile:
        uid2path = pickle.load(outfile)


In [41]:
sim_arts = []
for i, artwork in enumerate(madonnas.iterrows()):
    if artwork[1]['uid'] in reverse_map.values():
        sim_arts.append(find_most_similar_orig(artwork[1]["uid"], tree, embeddings, reverse_map, n=6))

            

In [42]:
sim_arts = list(set([x for xx in sim_arts for x in xx]))
print(len(sim_arts))
sim_arts

925


['df2b7e6102ac4c9aac922bf1a31b3ee9',
 '9c64efe6c48e412ba37259d282087112',
 'd47e4e4a4c2046d29375e9748477378c',
 '0fbe3d9e6fa1492e8151a7c990b7d22a',
 'b5b9eed8509e421480f10018bfdb6c8b',
 '22eca94deedd4eabaff95af87b8ceb77',
 'fd7281b5adf14637956ea67317be55dd',
 '5ef9a1c3da7d46d9856172f0dedc0bdd',
 'f2a6d0f33cb84be7a87ea087bc63e7b6',
 'bf71d636f40b42b88cb2f3b5463e1cbe',
 'f130505a2c824af8b9dcd305d1a2181b',
 'c87a357812d247429067c66d0509edc0',
 '332e386bf232426abdddf2e97d7ef9dd',
 '5e3f7396f2fb4e14a21ffded1b8a8845',
 'c56208b38152404c8b4b42a89f3f9fc8',
 '8b6a207d3c7e49429a24b0426135af79',
 'cf35dc2e6d4e40ee940f12d6db3e33ef',
 '27253013d8374d1794524425be45c942',
 '4a7028f0a8a744b38eb51dd427c9f09d',
 'de400a863d3846fcabee3ac943208821',
 'fb23ee8d5fad4fcdb91202e084307578',
 '0a7c8744e2334117a5466d1530829aa2',
 '7d30562215754315844b332721041620',
 '7759ead5586b4beea645a40755130e3c',
 '5c199ad50a0b4bb7993718a224d15568',
 '88dab0e67c4442c28fd9f872e2d931ff',
 'f4c57e239ab44d6e9a201e590d56d2ca',
 

In [43]:
final = list(set(sim_arts + list(madonnas['uid'])))
final
print(len(final))

2654


In [49]:
with open(data_dir + 'list_iconography.pkl', 'wb') as oufile:
    pickle.dump(final, oufile)
    